In [49]:
!pip install spotipy
!pip install geopandas
!pip install geopy
!pip install requests
!pip install lxml
!pip install pandas
!pip install numpy
!pip install gmplot

In [50]:
import requests
import spotipy
import geopandas
import geopy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
from lxml import html
import re
import pandas as pd
import numpy as np
import math
from geopy.geocoders import Nominatim

## Spotify Scraper

### Connect to Spotify API

In [51]:
client_id = '693faf0bce4f4fb0aa5fc5d8b3cc0b25'
client_secret = 'c2ab38a172f44774a0cfae1d7734130f'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

### Scraper Functions
get_artistID_by_artistName - check for reliability, better search

In [52]:
#not deemed reliable yet
def get_artistID_by_artistName(artistName):
        artist = sp.search(q=artistName, type="artist")
        #print(artist)
        artistObject = artist['artists']['items']
        if len(artistObject) == 0:
            return "NOT IN SPOTIFY"
        else:
            artistID = artistObject[0]['id']
            #print('AID: ', artistID)
            return artistID;

def get_genre(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain genre
    genres = artist['genres']
    if len(genres) == 0:
        return 'NONE'
    else:
        return genres

def get_followers(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain follower count
    followers = int(artist['followers']['total'])
    return followers

def get_monthlyListeners(artistID):
    r = requests.get('https://open.spotify.com/artist/{}'.format(artistID), timeout=10)
    match = re.search(r'Monthly Listeners', r.text)
    print(r.text)
    #get monthly listeners starting point
    string = r.text[match.start():]
    #split at colon
    strings = string.split(':')
    strings1 = strings[1].split(',')
    monthlyListeners = strings1[0].split(",")
    #grab second string
    return int(monthlyListeners[0].lstrip())

def get_cities(artistID):
    r = requests.get('https://open.spotify.com/artist/{}'.format(artistID), timeout=10)
    match = re.search(r'Where People Listen', r.text)
    #get where people listen starting point
    string = r.text[match.start():]
    #split at colon
    strings = string.split(':')
    strings1 = strings[1].split('"')
    cities = strings1[0].split(",")
    #grab second string
    return cities

## Songkick Scraper

### Connect to Songkick API

In [53]:
# Define API Key
api_key = "FYSeGSNlg6HXIfAr"

### Scraper Functions

get_venueArtists: add in festival/lineup capabilities

get_venueGenres: clean up mapping

get_venueSize: Decide how to use followers/listeners, fix metric (within the 30-60 percentile?), regression using Songkick Venue Capacity?

In [54]:
def get_venueArtists(metroID):
    # request json
    r = requests.get("https://api.songkick.com/api/3.0/metro_areas/"+ str(metroID) +"/calendar.json?apikey=" + str(api_key))
    # parse for events
    events = r.json()["resultsPage"]["results"]["event"]
    # define dictionary for storage
    venue_artist_dict = {}
    #loop through events
    for i in range(len(events)):
        # The artist
        artist = events[i]["performance"][0]["artist"]["displayName"]
        # Venue Name
        venue = events[i]["venue"]["displayName"]
       # venueLat = events[i]["venue"]["lat"]
        # venueLong = events[i]["venue"]["lng"]
        # if the venue is not already in dict, add it with the artist as its value
        if venue not in venue_artist_dict.keys():
            venue_artist_dict[venue] = [artist]
        # if it is in the dict, add the act to its 
        else:
            if artist not in venue_artist_dict[venue]:
                venue_artist_dict[venue].append(artist)
    return venue_artist_dict

def get_venueGenres(venue_artist_dict):
    venue_genre_dict = {}
    # loop through venues
    for venue in venue_artist_dict.keys():
        # loop through artists at each venue
        genre_list = []
        for artist in venue_artist_dict[venue]:
            #get artist id
            id_return = get_artistID_by_artistName(artist)
            # if its too small of an artist, ignore it
            if id_return == 'NOT IN SPOTIFY':
                artist_genre = 'NONE'
                # get the genre of artists in spotify
            else:
                artist_genre = get_genre(id_return)
                print
                mapped_genre=-1
                # replace the genre with a code... NEED TO CLEAN UP
                if any("funk" in s for s in artist_genre):
                    mapped_genre = 0
                if any("rock" in s for s in artist_genre):
                    mapped_genre = 1 
                if any("punk" in s for s in artist_genre):
                    mapped_genre = 1
                if any("reggae" in s for s in artist_genre):
                    mapped_genre = 2
                if any("r&b" in s for s in artist_genre):
                    mapped_genre = 3
                if any("soul" in s for s in artist_genre):
                    mapped_genre = 3
                if any("rap" in s for s in artist_genre):
                    mapped_genre = 4
                if any("bounce" in s for s in artist_genre):
                    mapped_genre = 4
                if any("pop" in s for s in artist_genre):
                    mapped_genre = 5
                if any("jazz" in s for s in artist_genre):
                    mapped_genre = 6
                if any("electronic" in s for s in artist_genre):
                    mapped_genre = 7
                if any("indie" in s for s in artist_genre):
                    mapped_genre = 8
                if any("country" in s for s in artist_genre):
                    mapped_genre = 9
                if any("cajun" in s for s in artist_genre):
                    mapped_genre = 9
                # add genre to list
                genre_list.append(mapped_genre)
        # assign genres to venue
        venue_genre_dict[venue] = list(set([x for x in genre_list if x != -1]))
    # return dictionary
    return venue_genre_dict

def get_venueSize(venue_artist_dict):
    venue_size_dict = {}
    # loop through venues
    for venue in venue_artist_dict.keys():
        # loop through artists at each venue
        size_list = []
        for artist in venue_artist_dict[venue]:
            #get artist id
            id_return = get_artistID_by_artistName(artist)
            # if its too small of an artist, its size is basically 0
            if id_return == 'NOT IN SPOTIFY':
                size_list.append(0)
            # get their followers
            else:
                artist_followers = get_followers(id_return)
                size_list.append(artist_followers)
        venue_size_dict[venue] = np.mean(np.array(size_list))
    return venue_size_dict

## Matching Functions
Both: Need to figure out a way to clean it up


In [55]:
#
# FUNCTION TO ADD ELEMENTS TO DICT
#
def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], tuple):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = value

#
# GENRE MATCH FUNCTION
# if artist genre identity is a match for venue genre, match (hard constraints)
#

def genre_match(d, genre_of):
    #print('in genre check function')
    genre_filtered_dictionary = {}

    # iterate through artist's given constraints O(n)
    for i in genre_of:
        for key, value in d.items():
            for j in value:
                if i == j:
                    append_value(genre_filtered_dictionary, key, value)
                else:
                    pass

    return genre_filtered_dictionary

def size_match(d,size_of):
    perfect_match_dict={}
    for key,value in d.items():
        num=value
        difference=abs(num-size_of)
        per=difference/size_of
        percentage1=1-per
        if(percentage1>=.4):
            perfect_match_dict[key]= round(percentage1, 2)
            None
    return perfect_match_dict

## New Orleans Example

In [57]:
#use names from class... THIS PART WILL BE CHANGED!

names = ["Vampire Weekend","The Doobie Brothers", "Greta Van Fleet", "Tank and the Bangas"]

df = pd.DataFrame({"band_name": names})
df["id"] = df.band_name.apply(get_artistID_by_artistName)
df["genre"] = df.id.apply(get_genre)
#df["listeners"] = df.id.apply(get_monthlyListeners)
df["followers"] = df.id.apply(get_followers)
#pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth',None)
df['mapped_genre'] = [[] for _ in range(df.shape[0])]
for i in range(df.genre.size):
    if any("funk" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('0')
        
    if any("rock" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('1')
        
    if any("punk" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('1')
        
    if any("reggae" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('2')
        
    if any("r&b" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('3')
        
    if any("rap" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('4')
        
    if any("pop" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('5')
        
    if any("jazz" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('6')
        
    if any("electronic" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('7')
        
    if any("indie" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('8')
        
    if any("country" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('9')
df

,band_name,id,genre,followers,mapped_genre
0,Vampire Weekend,5BvJzeQpmsdsFp4HGUYUEx,"[baroque pop, indie pop, indie rock, modern ro...",1689582,"[1, 5, 8]"
1,The Doobie Brothers,39T6qqI0jDtSWWioX8eGJz,"[album rock, art rock, blues rock, classic roc...",1473095,"[1, 9]"
2,Greta Van Fleet,4NpFxQe2UvRCAjto3JqlSl,"[modern blues rock, modern hard rock, rock]",1358172,[1]
3,Tank and the Bangas,5cAtakaadWHJLxmGKrKcX7,"[indie soul, new orleans funk]",156536,"[0, 8]"


In [58]:
#
# OUTPUT SECTION
# change this portion when looking at different bands
#
nola = get_venueArtists("11772")
nola_venue_genres = get_venueGenres(nola)
nola_venue_sizes = get_venueSize(nola)
column_names = ["Artist", "Artist Genre", "Artist Size", "Genre Matches", "Size Matches"]
output_df = pd.DataFrame(columns = column_names)

for i in range(0,len(df)):
    #get artist genres
    artist_genre = []
    for j in range(0,len(df.mapped_genre[i])):
        artist_genre.append(int(df.mapped_genre[i][j]))

    #get artist size
    artist_size = df.followers[i]

    #calculate genre matches
    genre_matches = genre_match(nola_venue_genres ,artist_genre)
    
    
    #calculate size matches
    size_matches = size_match(nola_venue_sizes ,artist_size)
    output_df = output_df.append({'Artist':df.band_name[i],'Artist Genre':artist_genre,
                                  'Artist Size':artist_size,'Genre Matches':genre_matches,
                                  'Size Matches':size_matches}, ignore_index=True)

output_df_styled = output_df.style.set_table_styles([dict(selector="th",props=[('max-width', '50px')])])
output_df_styled

,Artist,Artist Genre,Artist Size,Genre Matches,Size Matches
0,Vampire Weekend,"[1, 5, 8]",1689582,"{'Unknown venue': [1], 'Fillmore New Orleans': [[8, 1], [8, 1]], 'Smoothie King Center': [[1, 5, 9], [1, 5, 9]], 'UNO Lakefront Arena': [[8, 1], [8, 1]], 'Civic Theatre': [5], 'Saenger Theatre New Orleans': [4, 5], 'Orpheum Theater': [9, 5], 'House of Blues - New Orleans presented by Cricket Wireless': [8, 4], 'One Eyed Jacks': [8, 4], ""Tipitina's"": [8], 'Southport Hall': [8]}","{'Fillmore New Orleans': 0.77, 'Saenger Theatre New Orleans': 0.86, 'Orpheum Theater': 0.58}"
1,The Doobie Brothers,"[1, 9]",1473095,"{'Unknown venue': [1], 'Fillmore New Orleans': [8, 1], 'Smoothie King Center': [[1, 5, 9], [1, 5, 9]], 'UNO Lakefront Arena': [8, 1], 'Music Box Village': [9], 'Orpheum Theater': [9, 5]}","{'Fillmore New Orleans': 0.89, 'Saenger Theatre New Orleans': 0.98}"
2,Greta Van Fleet,[1],1358172,"{'Unknown venue': [1], 'Fillmore New Orleans': [8, 1], 'Smoothie King Center': [1, 5, 9], 'UNO Lakefront Arena': [8, 1]}","{'House of Blues - New Orleans presented by Cricket Wireless': 0.43, 'Fillmore New Orleans': 0.96, 'Saenger Theatre New Orleans': 0.93}"
3,Tank and the Bangas,"[0, 8]",156536,"{'Broadside': [0], 'Joy Theater': [0], 'House of Blues - New Orleans presented by Cricket Wireless': [8, 4], 'One Eyed Jacks': [8, 4], 'Fillmore New Orleans': [8, 1], ""Tipitina's"": [8], 'UNO Lakefront Arena': [8, 1], 'Southport Hall': [8]}","{'One Eyed Jacks': 0.41, 'Music Box Village': 0.96, 'Southport Hall': 0.91}"


### Traveling Salesperson Problem/Algorithm (TSP) - Example with Tank and the Bangas

#### Get latitude and longitude of each venue recommend by genre match

In [59]:
value = output_df.iloc[3]["Genre Matches"]
value

{'Broadside': [0],
 'Joy Theater': [0],
 'House of Blues - New Orleans presented by Cricket Wireless': [8, 4],
 'One Eyed Jacks': [8, 4],
 'Fillmore New Orleans': [8, 1],
 "Tipitina's": [8],
 'UNO Lakefront Arena': [8, 1],
 'Southport Hall': [8]}

In [90]:
column_names = ["Venue", "Latitude", "Longitude"]
venue_info_df = pd.DataFrame(columns = column_names)
for key in value.keys():
    r = requests.get("https://api.songkick.com/api/3.0/search/venues.json?query={"+key+"}&apikey=" + str(api_key))
    venueLat = r.json()['resultsPage']['results']['venue'][0]['lat']
    venueLng = r.json()['resultsPage']['results']['venue'][0]['lng']
    #not including venues that have no data for now
    if(venueLat == None):
        venueLat = None
    else:
        venue_info_df = venue_info_df.append({'Venue':key,'Latitude':venueLat,'Longitude':venueLng}, ignore_index=True)
venue_info_df = venue_info_df.set_index("Venue")
venue_info_df

,Latitude,Longitude
Venue,,
Joy Theater,29.956010,-90.073980
House of Blues - New Orleans presented by Cricket Wireless,29.953160,-90.066170
One Eyed Jacks,29.957032,-90.064749
Fillmore New Orleans,29.933560,-90.070010
Tipitina's,29.917410,-90.100780
UNO Lakefront Arena,30.029010,-90.052270
Southport Hall,29.954075,-90.136384


#### Get Lat and Long of Starting Point

In [91]:
startingPoint = "2619 Palmer Avenue, New Orleans, LA"
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode(startingPoint)
startingLat = location.latitude
startingLng =location.longitude

#### Function to calculate distance between two venues based on lat and long

In [92]:

def distance_on_earth(lat1, long1, lat2, long2, radius=6378.388):
    """
    Compute distance between two points on earth specified by latitude/longitude.
    The earth is assumed to be a perfect sphere of given radius. The radius defaults
    to 6378.388 kilometers. To convert to miles, divide by 1.60934
    
    Reference
    ---------
    Adopted from John D. Cook's blog post: 
    http://www.johndcook.com/blog/python_longitude_latitude/
    """
    # Convert latitude and longitude to spherical coordinates in radians.
    degrees_to_radians = np.pi / 180.0

    # phi = 90 - latitude
    phi1 = (90.0 - lat1) * degrees_to_radians
    phi2 = (90.0 - lat2) * degrees_to_radians

    # theta = longitude
    theta1 = long1 * degrees_to_radians
    theta2 = long2 * degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.     
    cos = (np.sin(phi1) * np.sin(phi2)* np.cos(theta1 - theta2) + 
           np.cos(phi1) * np.cos(phi2))
    arc = np.arccos(cos)
    rv = arc * radius
    return rv

#### Get distances between each venue and create distance matrix

In [112]:
mile_in_km = 1.60934
distances = {}
for i in range(len(venue_info_df.index)):
    a = venue_info_df.index[i]
    distances[a] = {}
    for j in range(len(venue_info_df.index)):
        b = venue_info_df.index[j]
        if j == i:
            distances[a][b] = 0.
        elif j > i:
            distances[a][b] = distance_on_earth(venue_info_df['Latitude'][a],
                                                venue_info_df['Longitude'][a],
                                                venue_info_df['Latitude'][b],
                                                venue_info_df['Longitude'][b])
        else:
            distances[a][b] = distances[b][a]
distances = (pd.DataFrame(distances)/mile_in_km)

In [113]:
distances

,Joy Theater,House of Blues - New Orleans presented by Cricket Wireless,One Eyed Jacks,Fillmore New Orleans,Tipitina's,UNO Lakefront Arena,Southport Hall
Joy Theater,0.000000,0.507903,0.557734,1.571073,3.116136,5.214492,3.742444
House of Blues - New Orleans presented by Cricket Wireless,0.507903,0.000000,0.281054,1.375202,3.227987,5.312496,4.208664
One Eyed Jacks,0.557734,0.281054,0.000000,1.653982,3.489539,5.034796,4.298141
Fillmore New Orleans,1.571073,1.375202,1.653982,0.000000,2.156601,6.687634,4.223947
Tipitina's,3.116136,3.227987,3.489539,2.156601,0.000000,8.248915,3.314745
UNO Lakefront Arena,5.214492,5.312496,5.034796,6.687634,8.248915,0.000000,7.229391
Southport Hall,3.742444,4.208664,4.298141,4.223947,3.314745,7.229391,0.000000


In [114]:
!pip install python-tsp

#### Use python-tsp library to solve tsp with dynamic programming

In [116]:
import numpy as np
from python_tsp.distances import great_circle_distance_matrix
from python_tsp.exact import solve_tsp_dynamic_programming

arr = []
for i in range(len(venue_info_df.index)):
    arr.append([venue_info_df['Latitude'][venue_info_df.index][i],venue_info_df['Longitude'][venue_info_df.index][i]])
sources = np.array(arr)
sources

array([[ 29.95601  , -90.07398  ],
       [ 29.95316  , -90.06617  ],
       [ 29.957032 , -90.064749 ],
       [ 29.93356  , -90.07001  ],
       [ 29.91741  , -90.10078  ],
       [ 30.02901  , -90.05227  ],
       [ 29.9540748, -90.1363836]])

In [117]:
distance_matrix = great_circle_distance_matrix(sources)

In [118]:
distance_matrix

array([[    0.        ,   816.44208666,   896.54408143,  2525.46254479,
         5009.11436569,  8382.17001632,  6015.88902846],
       [  816.44208666,     0.        ,   451.78793766,  2210.60384899,
         5188.91151297,  8539.7095493 ,  6765.325962  ],
       [  896.54408143,   451.78793766,     0.        ,  2658.73614543,
         5609.349984  ,  8093.31260312,  6909.15893362],
       [ 2525.46254479,  2210.60384899,  2658.73614543,     0.        ,
         3466.68392101, 10750.21031987,  6789.89303834],
       [ 5009.11436569,  5188.91151297,  5609.349984  ,  3466.68392101,
            0.        , 13259.9316267 ,  5328.37237671],
       [ 8382.17001632,  8539.7095493 ,  8093.31260312, 10750.21031987,
        13259.9316267 ,     0.        , 11621.07222359],
       [ 6015.88902846,  6765.325962  ,  6909.15893362,  6789.89303834,
         5328.37237671, 11621.07222359,     0.        ]])

In [119]:
permutation, distance = solve_tsp_dynamic_programming(distance_matrix)
distance

32303.13369354075

In [120]:
#convert distance from km to miles
distance = distance/1609
distance

20.076528087968146

In [121]:
permutation

[0, 3, 4, 6, 5, 2, 1]

In [122]:
s = pd.Series(permutation)

In [123]:
mappingDict = {}
for i in range(len(s)):
    mappingDict[i] = venue_info_df.index[i]


In [124]:
s = s.map(mappingDict)

In [125]:
optimalPath = s.tolist()

### Tank and The Bangs Optimal Tour Path in New Orleans

##### optimal path

In [126]:
optimalPath

['Joy Theater',
 'Fillmore New Orleans',
 "Tipitina's",
 'Southport Hall',
 'UNO Lakefront Arena',
 'One Eyed Jacks',
 'House of Blues - New Orleans presented by Cricket Wireless']

##### total distance in miles

In [127]:
distance

20.076528087968146